# Part 1: Data Preperation and Exploration

In [ ]:
# import packages and libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
from scipy.stats import shapiro
%matplotlib inline

In [ ]:
# Read file
df = pd.read_csv("../input/obesity-levels/ObesityDataSet_raw_and_data_sinthetic.csv")

# About the Data

This data comes from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition+). This dataset include data for the estimation of obesity levels in individuals from the countries of Mexico, Peru and Colombia, based on their eating habits and physical condition.

### Notes:
<ul> 
    <li> This data is created via survery. However the data was imbalanced and records were synthasized using WEKA to simulate additional responses.
    <li> The synthasized data fits the distribution of the data but generates floats (ratio data) where ordinal and integer data were the original values in the survey.
    <li> The target variable is comprised of the variables height and weight. Height and weight are used to classisfy a persons BMI. The BMI measure was then used to classify each observation. 
</ul>

### Features & Descriptions

<table>
    <thead>
        <tr><th>Category</th><th>Feature Name</th><th>Description</th><th>Variable Type</th></tr>
    </thead>
    <tbody>
        <tr><td>Target Variable</td><td>NObesity</td><td>Based on BMI</td><td>Categorical</td></tr>
        <tr><td>Eating Habits</td><td>FAVC</td><td>Frequent consumption of high caloric food</td><td>Categorical</td></tr>
        <tr><td>Eating Habits</td><td>FCVC</td><td>Frequency of consumption of vegetables</td><td>Ordinal</td></tr>
        <tr><td>Eating Habits</td><td>NCP</td><td>Number of main meals</td><td>Ordinal</td></tr>
        <tr><td>Eating Habits</td><td>CAEC</td><td>Consumption of food between meals</td><td>Ordinal</td></tr>
        <tr><td>Eating Habits</td><td>CH20</td><td>Consumption of water daily</td><td>Ordinal</td></tr>
        <tr><td>Eating Habits</td><td>CALC</td><td>Consumption of alcohol</td><td>Ordinal</td></tr>
        <tr><td>Physical Conditioning</td><td>SCC</td><td>Calories consumption monitoring</td><td>Categorical</td></tr>
        <tr><td>Physical Conditioning</td><td>FAF</td><td>Pysical activity frequency</td><td>Ordinal</td></tr>
        <tr><td>Physical Conditioning</td><td>TUE</td><td>Time using technology devices</td><td>Ordinal</td></tr>
        <tr><td>Physical Conditioning</td><td>MTRANS</td><td>Transportation used</td><td>Categorical</td></tr>
        <tr><td>Physical Conditioning</td><td>SMOKE</td><td>Smokes Yes or No</td><td>Categorical</td></tr>
        <tr><td>Responder Charateristics</td><td>Family History with Overweight</td><td>Yes or No</td><td>Categorical</td></tr>
        <tr><td>Responder Charateristics</td><td>Gender</td><td>Gender is Male or Female</td><td>Categorical</td></tr>
        <tr><td>Responder Charateristics</td><td>Age</td><td>Age in years</td><td>Integer</td></tr>
        <tr><td>Responder Charateristics</td><td>Height</td><td>Height in meters</td><td>Float</td></tr>
        <tr><td>Responder Charateristics</td><td>Weight</td><td>Weight in kilograms</td><td>Float</td></tr>        
     </tbody>
</table>



# Exploring the Data & Clean Up

- The data contains 2111 records with 17 columns
- The data loads as text and float objects for most of the objects. However we know that some are float, categorical and ordinal
- All of the records are unique and contain no null values
- Height and Weight are included however they have a direct correlation to each other and our target variable
- The survey data is distinguishable from the synthazied data based on floats used for ordinal variables


In [ ]:
# Top 5 rows show survey data
df.head(5)

In [ ]:
# Bottom 5 rows show synthetic data
df.tail(5)

In [ ]:
# Additional rows showing synthetic data
df.iloc[[501,518,516]]

#### Data Prep / Clean Up

- Drop Height and Weight columns as they are used in the BMI calculation for our target variable
- Convert categorical variables to category instead of object/text
- Convert synthetic floats & floats to whole integers to better reprsent the ordinal data from orignal survey data

In [ ]:
# Height and weight are highly correlated and they directly correlate to the BMI calc used for the target
# Remove Height and Weight
df = df.drop(columns=['Height', 'Weight'])
print(df.shape)

In [ ]:
# no nulls 
df[df.isnull().any(axis=1)]

In [ ]:
# Convert object/text variables to category variables
columns = ["Gender", "family_history_with_overweight", "FAVC", "CAEC", "SMOKE", "SCC", "CALC", "MTRANS", "NObeyesdad"]

for col in columns:
    df[col] = df[col].astype('category')

In [ ]:
# function to interigate data after conversion
# provides min, max, unique counts
def variable_counts(columns, stage):

    if stage == 'pre':
        print("Pre Conversion to Integer")
    else:
        print("Post Conversion to Integer")

    for col in columns:    
        print("Variable:", col, "| Count Unique:",df[col].nunique(),"| Min: ", df[col].min(), "| Max: ",df[col].max())

In [ ]:
# Convert float variables to integer to the nearest inter
columns = ["FCVC", "NCP", "CH2O", "TUE", "FAF"]

# pre conversion countss
variable_counts(columns, 'pre')

# convert to int / nearest int value
for col in columns:
    #round to nearest whole number
    df[col] = round(df[col]).astype('int')  
    
# post conversion counts
print("")
variable_counts(columns, 'post')

In [ ]:
# confirm types
df.info()

In [ ]:
# review non synthetic are still the same
df.head()

## Data Intuition & Further Exploration

##### Categorical Variables
- The categorical variables are not gaussian
- Most of the categorical variables are bernoulli in nature
- The target variables based on synthetic process are fairly balanced

##### Ordinal Variables                                        
- Will be treated as nominal         
- Non are Gaussian distributed

##### Ratio Variable
- Age is the only ratio varible
- Is not Gaussian

##### Predictor Coorelations 
- The predictor variables not highly correlated
- Height and Weight have been removed for their correlation to each other and the target

##### Target Variable
- Is categorical with > 2 classes
- Is faily balanced in its distribution of weights

In [ ]:
# columns of interest
columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE',
           'SCC', 'CALC', 'MTRANS', 'NObeyesdad']

fig, ax = plt.subplots(3, 3, figsize=(15, 10))
for col, subplot in zip(columns, ax.flatten()):
    sns.countplot(df[col], ax=subplot)
    
    if col=="MTRANS":
        sns.countplot(df[col],ax=subplot)
        subplot.set_xticklabels(rotation=45, horizontalalignment='right', labels=df.MTRANS)        
        subplot.yaxis.label.set_text("Number of Records")
    elif col=="NObeyesdad":
        sns.countplot(df[col],ax=subplot)
        subplot.set_xticklabels(rotation=45, horizontalalignment='right', labels=df.NObeyesdad)  
        subplot.yaxis.label.set_text("Number of Records")
    else:
        sns.countplot(df[col],ax=subplot)  
        subplot.yaxis.label.set_text("Number of Records")
        
# show figure & plots
fig.suptitle("Categorigal Variables", fontsize=20)
plt.tight_layout(pad=5, w_pad=0.0, h_pad=1)
plt.show()

In [ ]:
# columns of interest
columns = ["FCVC", "NCP", "CH2O", "FAF", "TUE"]

fig, ax = plt.subplots(1, 5, figsize=(15, 4))
for col, subplot in zip(columns, ax.flatten()):
    sns.countplot(df[col], ax=subplot)
    subplot.yaxis.label.set_text("Number of Records")

# show figure & plots
fig.suptitle("Ordinal Variables", fontsize=20)
plt.tight_layout(pad=5, w_pad=0.7, h_pad=0.5)
plt.show()

In [ ]:
# ratio variable distribution 

fig = plt.figure(figsize = (16,5))

#distplot
ax1 = fig.add_subplot(121)
sns.distplot(df["Age"], kde=True)

#boxplot
ax1 = ax1 = fig.add_subplot(122)
sns.boxplot(df.Age)

# show figure & plots
fig.suptitle("Distribution of Numeric (Ratio) Variable", fontsize=20)
plt.tight_layout(pad=5, w_pad=0.5, h_pad=.1)
plt.show()

In [ ]:
# create figure
fig = plt.figure(figsize=(15, 5))

# add subplot for one row 2 graphs first postion
ax1 = fig.add_subplot(121)

# correlation data matrix
matrix = np.triu(df.corr())

# set title 
ax1.title.set_text("Coorelation Heatmap: Predictor Variables")

#define plot
sns.heatmap(df.corr(), 
                 mask=matrix,
                 annot = False,                 
                 fmt='.1g', 
                 cmap="YlGnBu", 
                 vmin=-1, vmax=1, center= 0,                 
                 square="True",
                 ax=ax1)

# add second subplot
ax2 = fig.add_subplot(122)

# rotate axis label
ax2.set_xticklabels(rotation=45, horizontalalignment='right', labels=df.NObeyesdad)

# Set title text
ax2.title.set_text("Weight Category Counts: Target Variable")

# define second plot
sns.countplot(x="NObeyesdad",                  
                 palette="Blues_d", 
                 order=df.NObeyesdad.value_counts().index,
                 ax = ax2,
                 data=df)

# labels for x and y
ax2.xaxis.label.set_text("Level Category")
ax2.yaxis.label.set_text("Number of Records")

# turn off top and right frame lines
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)

# show figure & plots
plt.tight_layout()
plt.show()

In [ ]:
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

#print highly correlated variables
print("Number of variables with > 0.95 correlation: ", len(to_drop))

# Data Intuition & Model Considerations

##### Data Intuion
- Models for this data need to cater to > 2 two classes
- Data is limited so model will need to perform well with limited amount of data
- Model will need to handle dummy variables well and not be dependent on ratio data
- The ordinal variables may or may not be helful to the model
- The data is not Gaussian so the model needs to be non-parametic or at least not strictly parametic 
- The data is pretty wide but not very deep, so reducing the factors may be necessary

##### Potential Models
- Decision Trees 
- Random Forest 
- Search Vector Machines (SVM)

##### Experiments 
Models that may be used to evaluate data assumptions and model performance:
- Is the data too limited for a Nerual Network? 
- Does Logistic Regression perform well if target variable reduced to 2 classes?
- Do the other models perform better if target variable is reduced to 2 classes?

# Part 2: Machine Learning

The target variable is NObesity. The attempt is is apply ML to find the best model for predicting NObesity. 
NObesity is a categorical variable that is a measure of a person weight ranging from under weight to very obese (Overweight Level II). 

# Data prep for ML models: General 
- Some models may require additional treatment 
- These steps will prep the data for the most viable models: Decision Trees, Random Forest, SVM, Nerual Network
- Some models may benifit from scaling so this will be evaluated as well

#### Data Treatment
- Copy cleaned data to new dataframe
- Create dummy variables out of categorical variables
- Split the data into 70/30 train & test datasets     

In [ ]:
df_prep = df.copy()

In [ ]:
# create dummy variables
df_prep = pd.get_dummies(df_prep,columns=["Gender","family_history_with_overweight",
                                          "FAVC","CAEC","SMOKE","SCC","CALC","MTRANS"])
df_prep.head()

In [ ]:
# split dataset in features and target variable

# Features
X = df_prep.drop(columns=["NObeyesdad"])

# Target variable
y = df_prep['NObeyesdad'] 

In [ ]:
# import sklearn packages for data treatments
from sklearn.model_selection import train_test_split # Import train_test_split function

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

# Machine Learning Models

For this exercise we will take a look at 
- Decision Trees
- Random Forest
- Support Vector Machines (SVM)
- K Nearest Neighbors 

Cursory Look at the model results suggest that Random Forest will be our best initial model with an accuracy of 79%. However, many of the other models performance is not too far off. 

The other interesting result is that all of the models classify Obesity_Type_III amazingly well with >= 98% accuracy across all models. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.preprocessing import StandardScaler # Import for standard scaling of the data
from sklearn.preprocessing import MinMaxScaler # Import for standard scaling of the data

# standard scale data
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

# tested MinMaxScaler as KNN historically does better with MinMax
mm = MinMaxScaler()
X_train_mm_scaled = ss.fit_transform(X_train)
X_test_mm_scaled = ss.transform(X_test)

# program to run multilple models though sklearn 
# Default settings output accuracy and classification report
# compares accuracy for scaled and unscaled data
def run_models(X_train: pd.DataFrame , y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame):
    
    models = [          
          ('Random Forest', RandomForestClassifier(random_state=2020)),
          ('Decision Tree', DecisionTreeClassifier()),                                                 
          ('KNN', KNeighborsClassifier()),
          ('SVM', SVC())
        ]  
    
    for name, model in models:        
        # unscaled data
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        # scaled data
        clf_scaled = model.fit(X_train_scaled, y_train)
        y_pred_scaled = clf_scaled.predict(X_test_scaled)
        
        # mm scaled data
        clf_mm_scaled = model.fit(X_train_mm_scaled, y_train)
        y_pred_mm_scaled = clf_scaled.predict(X_test_mm_scaled)
        
        # accuracy scores
        accuracy = round(metrics.accuracy_score(y_test, y_pred),5)
        scaled_accuracy = round(metrics.accuracy_score(y_test, y_pred_scaled),5)
        scaled_mm_accuracy = round(metrics.accuracy_score(y_test, y_pred_mm_scaled),5)
        
        # output
        print(name + ':')        
        print("---------------------------------------------------------------")      
        print("Accuracy:", accuracy)
        print("Accuracy w/Scaled Data (ss):", scaled_accuracy)
        print("Accuracy w/Scaled Data (mm):", scaled_mm_accuracy)
        if (accuracy > scaled_accuracy) and (accuracy > scaled_mm_accuracy):
            print("\nClassification Report:\n", metrics.classification_report(y_test, y_pred))      
            print("                            -----------------------------------               \n")      
        elif (scaled_accuracy > scaled_mm_accuracy):
            print("\nClassification Report (ss):\n", metrics.classification_report(y_test, y_pred_scaled))      
            print("                            -----------------------------------               \n")     
        else:            
            print("\nClassification Report (mm):\n", metrics.classification_report(y_test, y_pred_mm_scaled))      
            print("                            -----------------------------------               \n")      

In [ ]:
#run Decision Trees, Random Forest, KNN and SVM
run_models(X_train, y_train, X_test, y_test)

# Hyperparameter Tuning

Searching for better performance out of the models with Gridsearch

In [ ]:
from sklearn.model_selection import GridSearchCV

#model name, classifier, parameters
# function used to process models and parameters through gridsearch
def hyper_tune(name, clf, parameters, target_names=None): 
    
    target_names = target_names
    clf = clf
    search = GridSearchCV(clf, parameters,verbose=True, n_jobs=15, cv=5)
    search.fit(X_train_scaled,y_train)
    y_pred_scaled = search.predict(X_test_scaled)
    print ("Accuracy Score = %3.2f" %(search.score(X_test_scaled,y_test)))
    print (search.best_params_)
    print("\nClassification Report:\n", metrics.classification_report(y_test, y_pred_scaled, target_names=target_names))
    

In [ ]:
#the KNN model performs better on the unscaled data this function
# function for unscaled data
#model name, classifier, parameters
# function used to process models and parameters through gridsearch
def hyper_tune2(name, clf, parameters, target_names=None): 
    
    target_names = target_names
    clf = clf
    search = GridSearchCV(clf, parameters,verbose=True, n_jobs=15, cv=5)
    search.fit(X_train,y_train)
    y_pred = search.predict(X_test)
    print ("Accuracy Score = %3.2f" %(search.score(X_test,y_test)))
    print (search.best_params_)
    print("\nClassification Report:\n", metrics.classification_report(y_test, y_pred, target_names=target_names))

## KNN

In [ ]:
# Number of neighbors
n_neighbors = [int(x) for x in range(4, 15)]
# weights
weights = ['uniform','distance']
# distance metric
metric = ['euclidean', 'manhattan', 'chebyshev']
# computation algorithm
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
# power paramter
p=[1,2]

parameters = { 'n_neighbors': n_neighbors,
              'weights':weights,
              'metric':metric,
              'p':p,
              'algorithm': algorithm              
               }

hyper_tune2('KNN', KNeighborsClassifier(), parameters)

## Random Forest

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in range(10, 200,10)]
# Criterion
criterion = ['gini','entropy']
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in range(10, 100, 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in range(2, 5)]
# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in range(2, 5)]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# random state
random_state = [1010]

parameters = { 'criterion':criterion,
               'n_estimators': n_estimators,
              'max_depth':max_depth,
              #'random_state': random_state,
              #'max_features':max_features,
              #'min_samples_split':min_samples_split             
               }


hyper_tune('Random Forest',
           RandomForestClassifier(), parameters)

# Feature Importance w/ Random Forest

In [ ]:
# Create Decision Tree classifer object with optimized parameters
clf = RandomForestClassifier(criterion='entropy',
               n_estimators=52,
              max_depth = 51,              
              max_features='auto',
              min_samples_split=2,
              random_state=1010)

# Train Decision Tree Classifer
clf = clf.fit(X_train_scaled,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test_scaled)


In [ ]:
print(X.columns)

In [ ]:
feature_imp = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
fig = plt.figure(figsize=(10, 5))

# Creating a bar plot
sns.barplot(x=feature_imp.index, y=feature_imp)

# Add labels to your graph
plt.xticks(rotation=45, horizontalalignment='right')

plt.tight_layout()
plt.show()

# create features list
features_list = X.columns
features_list = features_list.tolist()

# Get numerical feature importances
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
print("\nTop 10 Features:")
display_top = [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances[:10]]

# Sort the feature importances by least important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = False)
# Print out the feature and importances 
print("\nBottom 10 Features:")
display_bottom = [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances[:10]]

# Exploring Two Category Targert Variable

### Data Prep

![](http://)Underweight mapped to being not obese. Arguably however it is just as much of a health concern as being overweight. 

In [ ]:
# map values 
weight_map = { 'Normal_Weight':0, 'Overweight_Level_I':0,
               'Overweight_Level_II':0, 'Obesity_Type_I':1,
               'Obesity_Type_II':1, 'Obesity_Type_III':1, 'Insufficient_Weight':0}

# map values
df_prep['weight_cat'] = df_prep['NObeyesdad'].map(weight_map)

In [ ]:
sns.countplot(x="weight_cat",                  
                 palette="Blues_d", 
                 order=df_prep["weight_cat"].value_counts().index,                 
                 data=df_prep)


# show figure & plots
plt.tight_layout()
plt.show()

In [ ]:
# split dataset in features and target variable

# Features
X = df_prep.drop(columns=["NObeyesdad","weight_cat"])

# Target variable
y = df_prep['weight_cat'] 

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

# Scaled version of X train and X test
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

> ## Random Forest with Two Category Target Variable

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in range(10, 200,10)]
# Criterion
criterion = ['gini','entropy']
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in range(10, 100, 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in range(2, 20,2)]
# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in range(2, 20, 2)]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# random state
random_state = [1010]

target_names = ['Not Obese', 'Obese']

parameters = { 'criterion':criterion,
               'n_estimators': n_estimators,
              'max_depth':max_depth,
              'random_state': random_state,
              'max_features':max_features
              #'min_samples_split':min_samples_split             
               }

hyper_tune('Random Forest', RandomForestClassifier(), parameters, target_names=target_names)

## Feature Importance with Random Forest: Two Category Variable Target

In [ ]:
# Create Random Forest classifer object with optimized parameters
clf = RandomForestClassifier(criterion='gini',
               n_estimators=110,
              max_depth = 20,              
              max_features='auto',              
              random_state=1010)

# Train Random Forest classifer
clf = clf.fit(X_train_scaled,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test_scaled)

In [ ]:
feature_imp = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
fig = plt.figure(figsize=(12, 5))

# Creating a bar plot
sns.barplot(x=feature_imp.index, y=feature_imp)

# Add labels to your graph
plt.xticks(rotation=45, horizontalalignment='right')

plt.tight_layout()
plt.show()

# create features list
features_list = X.columns
features_list = features_list.tolist()

# Get numerical feature importances
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
print("\nTop 10 Features:")
display_top = [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances[:10]]

# Sort the feature importances by least important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = False)
# Print out the feature and importances 
print("\nBottom 10 Features:")
display_bottom = [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances[:10]]